In [1]:

# !@_purpose: adjust the variables of the custom and stock scripts to be specific to their purpose -> integrate the custom script into the stock script
import pandas as pd
import os

# === Input parameters ===
months           = 12
custom_ad_spend  = 400      # $ you plan to spend on ads per month !@_todo change custom_ad_spend to custom_ad_spend
custom_cpc_avg          = 2.8       # $ per click (from our keyword analysis) !@_todo custom_cpc_avg to custom_cpc_avg
custom_conversion_rate  = 0.02      # 2% of clicks turn into orders !@_todo change custom_conversion_rate to custom_custom_conversion_rate

custom_rev_average = 3500      # $ per job  !@_todo avg_revenue to custom_revenue_avg
custom_hours_per_job    = 24        # your shop hours/job !@_todo hours_per_job to custom_custom_hours_per_job
hours_capacity   = 40 * 4.3  # 40 h/week × 4.3 weeks ≈ 172 h/mo
capacity_jobs    = int(hours_capacity // custom_hours_per_job)  # max jobs you can run

# === derive monthly demand from ad economics ===
custom_clicks_monthly       = custom_ad_spend / custom_cpc_avg                # !@_todo custom_clicks_monthly to custom_clicks_month
custom_jobs  = int(custom_clicks_monthly * custom_conversion_rate)    # !@_todo monthly_demaqnd_jobs to custom_jobs

# === build the forecast ===
rows = []
design_pipeline = []
build_backlog = jobs_in_build_last_month = 0

for month in range(1, months + 1):
    # 1. new orders from marketing
    new_orders = custom_jobs
    design_pipeline.append(new_orders)

    # 2. after 2-month design lag, move into build queue
    if len(design_pipeline) > 2:
        build_backlog += design_pipeline[-3]

    # 3. start as many builds as capacity allows
    jobs_started = min(capacity_jobs, build_backlog)
    build_backlog -= jobs_started

    # 4. complete the ones you started last month
    jobs_completed = jobs_in_build_last_month
    jobs_in_build_last_month = jobs_started

    # 5. cash: 50% upfront deposit at start, 50% on completion
    deposit_cash = jobs_started   * custom_rev_average * 0.5
    final_cash   = jobs_completed * custom_rev_average * 0.5
    total_cash   = deposit_cash + final_cash

    rows.append([
        month, new_orders, build_backlog,
        jobs_started, jobs_completed,
        deposit_cash, final_cash, total_cash
    ])

# === assemble DataFrame ===
cols = [
    "Month", "New Orders", "Build Backlog End",
    "Jobs Started", "Jobs Completed",
    "Deposit Cash ($)", "Final Cash ($)", "Cash In ($)"
]
df = pd.DataFrame(rows, columns=cols)

# === save to CSV ===
file_path = r"C:\users\aboba\documents\projects\three_month_cash_flow_projection.csv"
df.to_csv(file_path, index=False)

print(f"Forecast saved to {file_path}")


Forecast saved to C:\users\aboba\documents\projects\three_month_cash_flow_projection.csv
